In [3]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression

cancer = load_breast_cancer()

cancerDF = pd.DataFrame(data=cancer.data, columns = cancer.feature_names)
cancerDF['Class'] = cancer.target